# Overview

In this notebook we make map interactive showing locations of incidients. We want this map to be interactable (with java?) to highlight incidents with time periods,shout_tags, pager-codes.

# Data

In [28]:
import pandas as pd 
df = pd.read_csv("..\data\cleaned.csv")
# df.head()

In [29]:
df[['latitude', 'longitude']] = df['core_lat_long'].str.split(',', expand=True)
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

In [30]:

df[['latitude', 'longitude']].head()

,latitude,longitude
0,56.090115,-4.617764
1,56.302002,-4.720782
2,56.145450,-4.661916
3,56.016814,-4.611093
4,56.027043,-4.595379


In [31]:

#convert to float
df['latitude'] = df['latitude'].astype(float)


In [32]:

# df['longitude'] = df['longitude'].astype(float)



In [33]:

#show just these cols

feats=["core_lat_long", "latitude", "longitude"]

df[feats].head()

,core_lat_long,latitude,longitude
0,"56.0901146882635, -4.617763927445251",56.090115,-4.617764
1,"56.30200229959834, -4.720781936098027",56.302002,-4.720782
2,"56.14545006003698, -4.661915903592648",56.145450,-4.661916
3,"56.01681400723627, -4.611093212845342",56.016814,-4.611093
4,"56.027043486219625, -4.595378734373894",56.027043,-4.595379


In [39]:
# determine if there are NaN in df[feats].

df[feats].isnull().sum()

#drop those with nan
df = df.dropna(subset=feats)

In [40]:
df[feats].isnull().sum()


core_lat_long    0
latitude         0
longitude        0
dtype: int64

In [41]:
# Normalize cost data to range from 0 to 1 for heat map intensity
import folium
from folium.plugins import HeatMap

# could use aggregated_df to group data by repeated locations

# Create a map centered around Loch lomond
map_center = df[['latitude', 'longitude']].dropna().mean().tolist()

map = folium.Map(location=map_center, zoom_start=6)

# List of points for the heat map: lat, lng, and the weight
# heat_data = [
#     (row['latitude'], row['longitude'], row['pager_code']) 
#     for index, row in df.iterrows()
#     if pd.notna(row['latitude']) and pd.notna(row['longitude'])
# ]

heat_data = df[['latitude', 'longitude']].values.tolist()


In [42]:
# Add the heat map layer
HeatMap(heat_data).add_to(map)

In [43]:
# Save to an HTML file
map.save('heat_map.html')

# Later


In [ ]:

# Assume 'expanded_df' is already loaded and includes latitude and longitude
# Calculate the average cost
import plotly.graph_objects as go

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=expanded_df['longitude'],
    y=expanded_df['latitude'],
    z=expanded_df['average_cost'], #TODO pager color?
    mode='markers',
    marker=dict(
        size=10,
        color=expanded_df['average_cost'],    # Set color according to average cost
        colorscale='Viridis',                 # Color scale for visualization
        opacity=0.8
    ),
    text=expanded_df['Company']              # Hover text
)])

# Update plot layout
fig.update_layout(
    title='3D Plot of Companies by Location and Average Cost',
    scene=dict(
        xaxis_title='Longitude',
        yaxis_title='Latitude',
        zaxis_title='Average Cost'
    ),
    margin=dict(l=0, r=0, b=0, t=0)          # Tight layout
)

# Show the plot
fig.show()

# Optionally, save the plot to an HTML file
fig.write_html('3D_scatter_plot.html')

expanded_df.to_csv('processed_companies.csv', index=False)

